In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA

import random
import os
import shutil

In [2]:
def similarity_clasters(texts, clasters, bert_pooler_outputs, treshold=0.9):
    dict_position_claster = {}
    dict_position_text = {}
    for i in range(len(clasters)):
        dict_position_claster[i] = clasters[i]
        dict_position_text[i] = texts[i]
    similarity_matrix = sklearn.metrics.pairwise.cosine_similarity(bert_pooler_outputs) # вычисляем матрицу симилярности каждого вектора с каждым
    np.fill_diagonal(similarity_matrix, 0) # убираю диагональные элементы, это симилярность вектора с самим собой
    #print('similarity_matrix = ', similarity_matrix)
    #print(np.shape(similarity_matrix))
    
    str_num = -1
    list_of_data_clasters = [[] for i in range(np.shape(similarity_matrix)[1])]
    str_index_to_skip = []
    while str_num < len(texts) - 1:
        str_num = str_num + 1
        add = True
        if str_num in str_index_to_skip:
            continue
        else:
            for colomn_num in range(len(similarity_matrix[str_num])):
                colomn_element = similarity_matrix[str_num][colomn_num]
                if colomn_element > treshold:
                    if colomn_num in str_index_to_skip:
                        continue
                    else:
                        if add:
                            list_of_data_clasters[str_num].append(str_num)
                            add = False
                        list_of_data_clasters[str_num].append(colomn_num)
                        str_index_to_skip.append(colomn_num)

    #print('list_of_data_clasters = ', list_of_data_clasters)
    
    
    for i in (list_of_data_clasters):
        claster = []
        claster_texts = []
        for j in i:
            claster.append(dict_position_claster[j])
            claster_texts.append(dict_position_text[j])
        if claster:
            print('Кластер:')
            print(claster)
            print(claster_texts)
            print('\n')

In [3]:
def dict_of_data(path = 'test_data.csv'):
    with open(path, encoding="UTF-8") as fp:
        dict_text_claster = {}
        for line in fp:
            strings = line[:-1].split(';')
            dict_text_claster[strings[0]] = strings[1]
    return dict_text_claster

def texts_clasters(path = 'test_data.csv'):
    texts = []
    dict_text_claster = dict_of_data(path)
    for key in dict_text_claster.keys():
      texts.append(key)
    
    clasters = []
    for value in dict_text_claster.values():
      clasters.append(value)
    return texts, clasters

In [4]:
dict_text_claster = dict_of_data()
texts, clasters = texts_clasters()

#c = list(zip(texts, clasters))
#random.shuffle(c)
#texts, clasters = zip(*c)

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(texts)

sim = sklearn.metrics.pairwise.cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings#[1:]
)

values, texts_sorted, clasters_sorted = zip(*sorted(zip(sim[0], texts, clasters), reverse = True))

for i in range(len(texts_sorted)):
    print(clasters_sorted[i] , ', ', texts_sorted[i])

1 ,  Сокращение расходов на обучение в компаниях
36 ,  Нет описания ролей и принципов
35 ,  Отсутствие карьерного видения для сотрудников
10 ,  Развитие крупных игроков в бизнес-образовании
49 ,  Нехватка ресурсов для развития
12 ,  Усиление гос образования
15 ,  Проблема появления нового игрока на рынке
35 ,  Нет политики назначения сотрудников
33 ,  Слишком много проектов одновременно
46 ,  Не хватает компетентных и мотивированных руководителей программ
41 ,  Недостаток человеческих ресурсов для реализации стратегии
34 ,  Разрыв между старой и новой командой
25 ,  Не описаны принципы функционирования экосистемы
8 ,  Запрет (ограничение) офлайн обучения
37 ,  Не определены приоритеты внутреннего развития
26 ,  Высокая неопределенность в экосистемных переговорах
37 ,  Отсутствие стратегического видения для всей команды
9 ,  Слабая отстройка от конкурентов
29 ,  Демотивированные сотрудники
16 ,  Сложно конкурировать с онлайн-школами в рамках коротких программ
13 ,  У ключевых конкуренто

In [6]:
#similarity_clasters(texts, clasters, sentence_embeddings, 0.8)

In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

#Encode all sentences
embeddings = model.encode(texts)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)


In [8]:
print("Top most similar pairs:")
for score, i, j in all_sentence_combinations[0:61]:
    print("{} \t {} \t {:.4f}".format(texts[i], texts[j], cos_sim[i][j]))

Top most similar pairs:
Неэффективное распределение ресурсов 	 Неэффективное использование ограниченного управленческого ресурса 	 0.9304
Низкий уровень мотивации (денежной и неденежной) для сотрудников 	 Низкий уровень мотивации (денежной и неденежной) для всех 	 0.9065
Низкий охват бренда АБВ 	 Низкий охват бренда 	 0.8989
Отсутствие карьерного видения для сотрудников 	 Отсутствие стратегического видения для всей команды 	 0.8699
Низкое качество инфраструктуры АБВ 	 Не работает ИТ инфраструктура 	 0.8684
Нет продукт овнеров корпоративных программ 	 Не хватает компетентных и мотивированных руководителей программ 	 0.8614
Нехватка ресурсов для развития 	 Нехватка ресурсов 	 0.8610
Проблема конкурентного продукта 	 Трудности с разработкой конкурентных программ 	 0.8608
Сложно конкурировать с онлайн-школами в рамках коротких программ 	 Онлайн-школы с огромными маркетинговых бюджетами забивают каналы 	 0.8488
Не работает ИТ инфраструктура 	 Нет денег на оперативную модернизацию инфраструк

In [9]:
def kMeans(texts):
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    corpus = texts.copy()
    corpus_embeddings = embedder.encode(corpus)

    # Perform kmean clustering
    num_clusters = 61
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_

    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(corpus[sentence_id])

    for i, cluster in enumerate(clustered_sentences):
        if len(cluster) > 1:
            print("Cluster ", i+1)
            print(cluster)
            cluster_gt = []
            for j in cluster:
                cluster_gt.append(dict_text_claster[j])
            print(cluster_gt)
            print("")

In [10]:
kMeans(texts)

Cluster  3
['Отсутствие карьерного видения для сотрудников', 'Отсутствие стратегии и единого ее понимания', 'Отсутствие стратегического видения для всей команды']
['35', '37', '37']

Cluster  4
['Очень низкие ставки преподавателей', 'Недостаток сильных преподавателей']
['29', '42']

Cluster  5
['Проблема конкурентного продукта', 'Трудности с разработкой конкурентных программ']
['20', '20']

Cluster  6
['Низкий охват бренда АБВ', 'Низкий охват бренда']
['23', '23']

Cluster  7
['Низкое качество инфраструктуры АБВ', 'Не работает ИТ инфраструктура', 'Нет денег на оперативную модернизацию инфраструктуры']
['58', '58', '59']

Cluster  8
['Низкая вовлеченность', 'Низкий уровень инвестиций в развитие']
['29', '49']

Cluster  9
['Высокая неопределенность в экосистемных переговорах', 'Нет продукт овнеров корпоративных программ', 'Не хватает компетентных и мотивированных руководителей программ']
['26', '40', '46']

Cluster  10
['Производительность системы ниже из-за отсутствия мотивационных прог

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = texts.copy()
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
#clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.25)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])


In [12]:
for i, cluster in clustered_sentences.items():
    if len(cluster) > 1:
        print("Cluster ", i+1)
        print(cluster)
        cluster_gt = []
        for j in cluster:
            cluster_gt.append(dict_text_claster[j])
        print(cluster_gt)
        print("")

Cluster  21
['Сокращение расходов на обучение в компаниях', 'Развитие крупных игроков в бизнес-образовании']
['1', '10']

Cluster  5
['Очень долгий цикл корп продаж', 'Нет сформулированного пакета ценностей для экосистемных партнёров', 'Нет стратегических целей']
['4', '27', '37']

Cluster  22
['Слабая отстройка от конкурентов', 'Не обновлена структура оп']
['9', '55']

Cluster  3
['Развитие корп университетов', 'Нехватка ресурсов для развития', 'Нехватка ресурсов', 'Нет резервов']
['11', '49', '50', '52']

Cluster  31
['Усиление гос образования', 'Проблема открытия границ']
['12', '17']

Cluster  9
['Сильные бренды у конкурентов', 'Демотивированные сотрудники', 'Слишком много проектов одновременно']
['14', '29', '33']

Cluster  43
['Сложно конкурировать с онлайн-школами в рамках коротких программ', 'Онлайн-школы с огромными маркетинговых бюджетами забивают каналы']
['16', '16']

Cluster  12
['Слабое присутствие в регионах', 'Культурная парадигма не соответствует']
['19', '31']

Cluste

In [13]:
def PCA_one_basis_for_all(bert_pooler_outputs):
    pca = PCA(n_components=2, svd_solver='full')
    pca.fit(bert_pooler_outputs)
    PCA_values = pca.transform(bert_pooler_outputs)
    return PCA_values

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(texts)

PCA_2D = PCA_one_basis_for_all(sentence_embeddings)

sim = sklearn.metrics.pairwise.cosine_similarity(
    [PCA_2D[0]],
    PCA_2D#[1:]
)

values, texts_sorted, clasters_sorted = zip(*sorted(zip(sim[0], texts, clasters), reverse = True))

for i in range(len(texts_sorted)):
    print(clasters_sorted[i] , ', ', texts_sorted[i])

1 ,  Сокращение расходов на обучение в компаниях
22 ,  Обесценивание
24 ,  Реализовать видение экосистемных продаж без увеличения штата
9 ,  Не используем в полной мере в продвижении фактические конкурентные преимущества
29 ,  Низкий уровень мотивации (денежной и неденежной) для всех
29 ,  Низкий уровень мотивации (денежной и неденежной) для сотрудников
12 ,  Усиление гос образования
2 ,  Снижение продаж в штуках
56 ,  Не выстроен бизнес-процесс
35 ,  Нет политики назначения сотрудников
28 ,  Выше ставки у других б/ш
53 ,  Не сбалансированы маркетинг и продажи
16 ,  Сложно конкурировать с онлайн-школами в рамках коротких программ
57 ,  Не введена система постоянных улучшений у всех участников бизнеса
33 ,  Слишком много проектов одновременно
14 ,  Сильные бренды у конкурентов
36 ,  Нет описания ролей и принципов
23 ,  Низкий охват бренда
29 ,  "Нет систем развития, обучения и мотивации (особенно нематериальной)"
29 ,  Производительность системы ниже из-за отсутствия мотивационных прогр

1.0000001